<a href="https://colab.research.google.com/github/fjadidi2001/Artificial_Intelligence_Learning/blob/master/Alzheimer's_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load your dataset
# Replace 'your_dataset.csv' with the path to your dataset
data = pd.read_csv('your_dataset.csv')

# Assume the last column is the target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print evaluation metrics
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load your dataset
# Replace 'your_time_series_dataset.csv' with the path to your dataset
data = pd.read_csv('your_time_series_dataset.csv')

# Assuming your data is in a long format with columns ['patient_id', 'time_step', 'feature_1', ..., 'feature_n', 'label']
# Pivot the data to get it in the right shape for an RNN (samples, time steps, features)
# This step will vary greatly depending on the specific format of your data
# Here, we'll provide a general outline

# Example data manipulation: Assume each patient has an equal number of time steps
# Reshape the data (this part will depend on your specific dataset structure)
patients = data['patient_id'].unique()
time_steps = len(data[data['patient_id'] == patients[0]])
num_features = len(data.columns) - 3  # Excluding patient_id, time_step, and label

X = []
y = []

for patient in patients:
    patient_data = data[data['patient_id'] == patient].sort_values('time_step')
    X.append(patient_data.iloc[:, 2:-1].values)  # Features
    y.append(patient_data.iloc[0, -1])  # Label

X = np.array(X)
y = np.array(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = StandardScaler()
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

# Build the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print evaluation metrics
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [2]:
!pip install numpy pandas scikit-learn tensorflow


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load your clinical data from ADNI
# Replace 'adni_clinical_data.csv' with the path to your ADNI clinical data file
data = pd.read_csv('adni_clinical_data.csv')

# Example of preprocessing (this will vary based on your specific data structure)
# Assume 'RID' is the patient identifier, 'VISCODE' is the visit code (time point), and 'DX' is the diagnosis label
# Other columns represent clinical features

# Pivot the data to get it in the right shape for an RNN (samples, time steps, features)
data_pivoted = data.pivot_table(index='RID', columns='VISCODE', values=[col for col in data.columns if col not in ['RID', 'VISCODE', 'DX']])
labels = data.drop_duplicates('RID')[['RID', 'DX']].set_index('RID')

# Combine pivoted data and labels
combined_data = data_pivoted.join(labels)

# Remove rows with missing values
combined_data.dropna(inplace=True)

# Extract features and labels
X = combined_data.iloc[:, :-1].values
y = combined_data.iloc[:, -1].values

# Convert labels to binary (e.g., 0 for CN (Cognitively Normal), 1 for AD (Alzheimer's Disease))
y = (y == 'AD').astype(int)

# Reshape X to 3D (samples, time steps, features)
num_patients = len(combined_data)
num_time_steps = len(data['VISCODE'].unique())
num_features = X.shape[1] // num_time_steps
X = X.reshape((num_patients, num_time_steps, num_features))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scalers = {}
for i in range(X_train.shape[2]):
    scalers[i] = StandardScaler()
    X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

# Build the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print evaluation metrics
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: 'adni_clinical_data.csv'